Download the Dataset
Download the Dataset from the following link:
https://data.world/crowdflower/sentiment-analysis-in-text

Reading the Dataset
Read and Prepare your dataset.
Calculate:
Vocabulary size; and
Maximum sequence length.
Divide your dataset into training and test set:
70% training set; and
30% Testing set.

In [1]:
import pandas as pd
import numpy as np
text_emotion = pd.read_csv('/Users/yongpengfu/Desktop/text_emotion.csv')
text_emotion.head(2)

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...


In [2]:
#Vocabulary size; and
#Maximum sequence length.
text_emotion.iloc[:, 3] =  text_emotion.iloc[:, 3].str.lower()
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
words = text_to_word_sequence(text_emotion.content.str.cat(sep=' '))
#tokenize input text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(words)
tokens = tokenizer.word_index
#get the vocabulary size
vocabulary_size = len(tokens) + 1
print(vocabulary_size)
#build sequences
sequences = tokenizer.texts_to_sequences(text_emotion.content)

48998


In [3]:
#encoding the output y of sentiment
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(text_emotion.sentiment)
print(y.shape)
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=vocabulary_size)
print(y.shape)

(40000,)
(40000, 48998)


In [5]:
#Maximum sequence length.
sequence_length = max(len(sequence) for sequence in sequences)
#pad at the beginning of each sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen = sequence_length, padding = 'pre')
#Divide your dataset into training and test set:
#70% training set; and
#30% Testing set.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(sequences, y, test_size = 0.3, random_state = 0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((28000, 37), (12000, 37), (28000, 48998), (12000, 48998))

Define your Recurrent Neural Network
Define a RNN with the following layers:
The input layer is an embedding layer with the following parameters:
The input dimension is the vocabulary size;
The output dimension is 10;
The input length is the maximum sequence length;
An LSTM layer with 128 units; and
An LSTM layer with 64 units.
A fully-connected layer with:
100 units;
Activation function: relu; and
A Dropout layer with 0.5 rate.
The output layer is a fully-connected layer with:
5 units; and
Activation function: softmax.

In [12]:
from tensorflow.keras.models import Sequential
model = Sequential()
from tensorflow.keras.layers import Embedding
model.add(Embedding(vocabulary_size, 10, input_length = sequence_length ))
from tensorflow.keras.layers import LSTM
model.add(LSTM(128))
from tensorflow.keras.layers import Dense
model.add(Dense(units = 100, activation = 'relu'))
from tensorflow.keras.layers import Dropout
model.add(Dropout(0.5))
model.add(Dense(units = vocabulary_size, activation = 'softmax'))

Choosing Hyper-Parameters
Build the network using the following parameters:
Optimizer: adam;
Loss Function: categorical_crossentropy;
Metrics: accuracy;
Batch size: 256; and
Epochs: 10.

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [14]:
model.fit(x_train, y_train, batch_size = 256, epochs = 10)

Epoch 1/10
110/110 [==============================] - 32s 290ms/step - loss: 3.8789 - accuracy: 0.1980
Epoch 2/10
110/110 [==============================] - 34s 306ms/step - loss: 2.1722 - accuracy: 0.2102
Epoch 3/10
110/110 [==============================] - 34s 312ms/step - loss: 2.1690 - accuracy: 0.2127
Epoch 4/10
110/110 [==============================] - 34s 313ms/step - loss: 2.1680 - accuracy: 0.2105
Epoch 5/10
110/110 [==============================] - 36s 324ms/step - loss: 2.1678 - accuracy: 0.2137
Epoch 6/10
110/110 [==============================] - 34s 306ms/step - loss: 2.1639 - accuracy: 0.2124
Epoch 7/10
110/110 [==============================] - 35s 319ms/step - loss: 2.1669 - accuracy: 0.2118
Epoch 8/10
110/110 [==============================] - 31s 280ms/step - loss: 2.1642 - accuracy: 0.2170
Epoch 9/10
110/110 [==============================] - 30s 276ms/step - loss: 2.1644 - accuracy: 0.2148
Epoch 10/10
110/110 [==============================] - 32s 289ms/step - l

In [15]:
loss, accuracy = model.evaluate(x_test, y_test)

375/375 [==============================] - 9s 24ms/step - loss: 2.1392 - accuracy: 0.2103
